# Finding Accommodation with MultiOn

In this recipe, we'll demonstrate how to scrape data from AirBnB and make a booking using MultiOn's Agent API in local mode. We'll be using `step` and `retrieve` to find accommodation for our desired location and dates.

## Step 1: Set up the environment

First, let's install the required libraries and set up the MultiOn client.

In [ ]:
%pip install multion --upgrade

In [2]:
from multion.client import MultiOn

client = MultiOn(
    api_key="YOUR_API_KEY" # Get your API key from https://app.multion.ai/api-keys
)

## Step 2: Create the accommodation search agent

Next, we will create an agent session with local mode enabled, which allows us to see the agent in action in our local browser with the MultiOn Chrome Extension. This agent will be responsible for searching for accommodation according to our specifications.

Make sure that the MultiOn Chrome Extension is installed and enabled (for more details, see [here](https://docs.multion.ai/learn/browser-extension)).

In [3]:
create_response = client.sessions.create(
    url="https://www.airbnb.com/",
    local=True
)

session_id = create_response.session_id

If you aren't logged in to AirBnB, make sure you do so. Now, we can continue from the same session using the `session_id` from the response. We will search for accommodation and use `step` until we get to the results page.

In [ ]:
status = "CONTINUE"

while status == "CONTINUE":
    step_response = client.sessions.step(
        session_id=session_id,
        cmd="Find accommodation options in Mountain View from July 20 to 30."
    )
    status = step_response.status

## Step 3: Scrape Listings

Once we are on the results page, we can start retrieving data with `retrieve`.

Since AirBnB paginates its results, we will have to navigate to the next page with `step` once we have scraped all listings on the current page. Conveniently, `retrieve` has an option to scroll down to the bottom while retrieving (`scroll_to_bottom`), which we will use to speed up the process. We can also enable `render_js` to get all image URLs.
In the interest of brevity, we will limit the maximum pages we scrape to 5.

In [ ]:
scraped_listings = []
has_more = True
page = 1

while has_more:
    retrieve_response = client.retrieve(
        session_id=session_id,
        cmd="Get all the accommodation listings with name, rating, location, price per night, listing URL, and image URL.",
        fields=["name", "rating", "location", "price", "listing_url", "image_url"],
        scroll_to_bottom=True,
        render_js=True
    )
    scraped_listings.extend(retrieve_response.data)
    print(f"Scraped page {page} with {len(retrieve_response.data)} listings")
    page += 1
    step_response = client.sessions.step(
        session_id=session_id,
        cmd="Click the 'Next' button to go to the next page."
    )
    has_more = "last page" not in step_response.message
    if page >= 5:
        break

print(f"Scraped {len(scraped_listings)} profiles:\n{scraped_listings}")

In [ ]:
scraped_listings

## Step 4: Book a Room


Now, we will `create` a new session and use `step` to book the first listing from our scraped list.

⚠️ If you're logged in and have a payment method set up, this will complete the reservation (and make the payment) on AirBnB!

In [ ]:
chosen_listing = scraped_listings[0]
url = "https://www.airbnb.com" + chosen_listing['listing_url']

create_response = client.sessions.create(
    url=url,
    local=True
)

session_id = create_response.session_id

status = "CONTINUE"

while status == "CONTINUE":
    step_response = client.sessions.step(
        session_id=session_id,
        url=url,
        cmd="Book this listing for July 20 to 30."
    )
    status = step_response.status